In [2]:
import torch
import open_clip
from PIL import Image
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"

# Chọn model mạnh hơn: ViT-g-14 (huấn luyện trên LAION-2B)cái 
model_name = "ViT-SO400M-16-SigLIP2-384"
pretrained = "webli"

# Load model + preprocess
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name=model_name,
    pretrained=pretrained,
    device=device
)

# Load tokenizer cho text
tokenizer = open_clip.get_tokenizer(model_name)


d:\Summer_2025\AIC\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def extract_image_features(image_path):
    image = Image.open(image_path).convert("RGB")
    
    image_input = preprocess(image).to(device).unsqueeze(0)
    
    with torch.no_grad():
        image_features = model.encode_image(image_input)
    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    
    return image_features.cpu().detach().numpy()

In [56]:
def _to_pil(img_like):
    if isinstance(img_like, str):
        with Image.open(img_like) as im:
            return im.convert("RGB").copy()
    elif isinstance(img_like, Image.Image):
        return img_like.convert("RGB").copy()
    else:
        raise TypeError(f"Unsupported type for image: {type(img_like)}")

In [ ]:
cu chay file quẻy trước di
vang anh
anh nghi? ngoi di a. Em ngoi lam lai anh ôi

In [4]:
def text_embed(text: str):

    ###### TEXT FEATURES EXTRACTING ######
    text = tokenizer([text]).to(device)
    text_features = model.encode_text(text)

    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().detach().numpy()

    return text_features


In [6]:
@torch.no_grad()
def embed_images(images):
    # pil_img = _to_pil(images)
    batch = torch.stack([preprocess(img) for img in images]).to(device)
    feats = model.encode_image(batch).float()
    return feats.cpu().numpy().astype("float16")

In [7]:
file_img = r'D:\Summer_2025\AIC\Data_extraction\Keyframes_2025\Keyframes_L21\keyframes\L21_V001\005.jpg'
imgs = Image.open(file_img).convert("RGB").copy()
test_img = embed_images([imgs])

In [8]:
text = 'a man sitting on a motorbike' # query voi text nay , chay ben file query cuxng duoc
# oke anh, de em ngoi lam anh a.
te = text_embed(text)

In [67]:
te.shape

(1, 1152)

In [8]:
import numpy as np
file = r'D:\Summer_2025\AIC\Data_extraction\features\L21_V001.npy'
a = np.load(file)

In [14]:
a[4]

array([ 0.006508,  0.0324  ,  0.00977 , ...,  0.005356, -0.0218  ,
        0.0423  ], dtype=float16)

In [17]:
a.shape

(307, 1152)

In [34]:
import faiss
dim = a.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(a)
index.ntotal

307

In [27]:
test = test.astype("float32")

In [28]:
test

array([ 0.00650787,  0.03240967,  0.00977325, ...,  0.00535583,
       -0.02180481,  0.04229736], dtype=float32)

In [29]:
test.shape

(1152,)

In [71]:
D, I = index.search(te, k=3) # neeus tra ve `38` laf ok 

In [72]:
D,I

(array([[0.11039707, 0.10580194, 0.10005827]], dtype=float32),
 array([[ 38, 165, 168]], dtype=int64))

In [ ]:
gio em check code cac phan lien quan toi embed_text been file query.py cua em nha 

copy khong sua logic , chay dung cau query anh de do, ra kq giong la ok
ok anh
ma tai sao lai. la` 38 v anh

gio chay query.py 
sao cho array tra ve 38 ha? anh
dung roi , 

In [ ]:
# index = faiss.read_index(INDEX_PATH)
# # tăng recall nếu cần
# faiss.ParameterSpace().set_index_parameter(index, "nprobe", 256)
import ujson
conn = sqlite3.connect(SQLITE_DB)

def search_vec(vec, k=10):
    D, I = index.search(vec, k)
    out = []
    for s, idx in zip(D[0].tolist(), I[0].tolist()):
        if idx == -1: continue
        row = conn.execute("SELECT path, payload FROM docs WHERE id=?", (int(idx),)).fetchone()
        if row:
            out.append({"id": int(idx), "score": float(s), "path": row[0], "payload": ujson.loads(row[1])})
    return out

In [78]:
CTX_LEN = int(model.text.positional_embedding.shape[0])

def tokenize_texts(texts):
    try:
        toks = open_clip.tokenize(texts, context_length=CTX_LEN, truncate=True)
    except TypeError:
        # Fallback cho bản open-clip cũ không có 'truncate'
        toks = open_clip.tokenize(texts, context_length=CTX_LEN)
        # đảm bảo đúng [B, CTX_LEN]
        if toks.shape[1] > CTX_LEN:
            toks = toks[:, :CTX_LEN]
        elif toks.shape[1] < CTX_LEN:
            pad = torch.zeros((toks.shape[0], CTX_LEN - toks.shape[1]), dtype=toks.dtype)
            toks = torch.cat([toks, pad], dim=1)
    return toks

In [1]:
import torch.nn.functional as F
USE_COSINE = False
@torch.no_grad()
def embed_text(texts):
    toks = tokenizer(texts).to(device)
    feats = model.encode_text(toks).float()
    if USE_COSINE:
        feats = F.normalize(feats, dim=-1)
    return feats.cpu().numpy().astype("float16")

NameError: name 'torch' is not defined

In [90]:
text_test = 'a man sitting on a motorbike'
te_test = embed_text(text_test)

In [91]:
te_test.shape

(1, 1152)

In [92]:
D, I = index.search(te_test, 3)

In [93]:
D,I

(array([[0.11039121, 0.10579748, 0.10005896]], dtype=float32),
 array([[ 38, 165, 168]], dtype=int64))